# 「都民のくらしむき」東京都生計分析調査報告（月報） 主要「指標１	全世帯」のデータ加工

## 参照サイト：「都民のくらしむき」東京都生計分析調査報告（月報）　令和２年10月
https://www.toukei.metro.tokyo.lg.jp/seikei/2020/sb20a20000.htm

## 利用データ
https://www.toukei.metro.tokyo.lg.jp/seikei/2020/sb20av0100.csv

## 加工方法（概要）
* 表部分, 年月に関する情報, 消費支出に関する明細項目のみ抽出
* 消費支出に関する明細項目を縦持ちに変更した上で年月に関する情報と結合

## 注意事項
* 元データを流用した列名は和名, 加工を加えて新規に追加した列名は英名としています
* 本コードの利用時はデータの取得元の情報を確認の上、自己責任でお願いいたします

In [1]:
import pandas as pd

In [2]:
infilename = './data/sb20av0100.csv'
outfilename = './data/sb20av0100_proc.csv'

In [3]:
# 表形式になっている6行目以降を読み取り
df = pd.read_csv('./data/sb20av0100.csv', encoding='SJIS', header=6)
df.head()

,7,実数・増減率,年次・四半期・月次コード,年次・四半期・月次,調査世帯数(世帯）,世帯人員（人）,有業人員（人）,世帯主平均年齢（歳）,生計支出（円）,生計支出／消費支出（円）,...,生計支出／消費支出／光熱・水道（円）,生計支出／消費支出／家具・家事用品（円）,生計支出／消費支出／被服及び履物（円）,生計支出／消費支出／保健医療（円）,生計支出／消費支出／交通・通信（円）,生計支出／消費支出／教育（円）,生計支出／消費支出／教養娯楽（円）,生計支出／消費支出／その他の消費支出（円）,生計支出／消費支出／その他の消費支出／諸雑費（円）,生計支出／土地家屋購入（円）
0,8,Jissu,nen,実数_平成28年平均,735,2.99,1.31,59.1,321733,320605.0,...,20563.0,11219.0,14495.0,15045.0,33439.0,19946.0,36360.0,60225,26043.0,1128
1,9,Jissu,nen,実数_平成29年平均,728,2.94,1.28,60.2,330949,327903.0,...,21024.0,12153.0,14135.0,15518.0,36746.0,19626.0,37292.0,62002,25956.0,3046
2,10,Jissu,nen,実数_平成30年平均,717,2.92,1.34,59.7,323328,322314.0,...,21383.0,10722.0,13654.0,16097.0,35656.0,20417.0,36071.0,58264,25614.0,1013
3,11,Jissu,nen,実数_平成31年・令和元年平均,703,2.92,1.31,60.8,330278,326563.0,...,21274.0,12296.0,14047.0,16717.0,36545.0,19284.0,37513.0,57353,25810.0,3716
4,12,Jissu,sihan,実数_令和元年 7〜 9月,710,2.95,1.31,60.7,334774,334774.0,...,17929.0,14801.0,12576.0,17240.0,38922.0,18714.0,39711.0,58788,26868.0,-


In [4]:
# 利用したい文字列項目を定義
tar_col_1  = [
 '実数・増減率',
 '年次・四半期・月次コード',
 '年次・四半期・月次'
]

In [5]:
# 利用したい数値項目を定義
tar_col_2  = [
 '生計支出／消費支出／食料（円）',
 '生計支出／消費支出／住居（円）',
 '生計支出／消費支出／光熱・水道（円）',
 '生計支出／消費支出／家具・家事用品（円）',
 '生計支出／消費支出／被服及び履物（円）',
 '生計支出／消費支出／保健医療（円）',
 '生計支出／消費支出／交通・通信（円）',
 '生計支出／消費支出／教育（円）',
 '生計支出／消費支出／教養娯楽（円）',
 '生計支出／消費支出／その他の消費支出（円）'
]

In [6]:
# 元の表から文字列項目の列のみ抽出
df_1 = df.loc[:, tar_col_1]
# のちほど結合のキーとするためにインデックスの値を保持
df_1['index_no'] = df_1.index

# 元の表から数値項目の列のみ抽出
df_2 = df.loc[:, tar_col_2]
# データを縦持ちに変換
df_2_st = df_2.stack().reset_index()
# カラムの意味がわかりやすいようにリネーム
df_2_st.rename(columns={'level_0': 'index_no',  'level_1': 'class', 0 : 'value'}, inplace=True)

In [9]:
result = pd.merge(df_1, df_2_st, on='index_no')
result = result.drop('index_no', axis=1)

In [10]:
result.to_csv(outfilename, index=False)